In this lesson, we'll force on `permutation importance`. Compared to most other approaches, permutation importance is:

 - Fast to calculate
 - Widely used and understood
 - Consistent with properties we would want a feature importance measure to have

**If I randomly shuffle a single column od the vaidation data, leaving the target and all other columns in place, how would that affect the accuracy of predictions in that now-shuddled data?**

with this insight, the process is as follows:

 1. Get a train model.
 2. Shuffle the values in a single columns, make predictions using the result dataset. Use these predictions and the true target values to calculate how much the loss function suffered from shuffing. That performance daterioration measure the importance of the variable you just shuffled.
 3. Return the data to the original order(undoing the shuffle from step 2). Now repeat step 2 with the next column in the dataset, until you have calculated the importance of each column.

**Code Example**

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('FIFA 2018 Statistics.csv')
y = (data['Man of the Match'] == 'Yes') #Convert from string "Yes"/"No" to binary
feature_names = [i for i in data.columns if data[i].dtype in [np.int64]]
X = data[feature_names]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)
my_model = RandomForestClassifier(n_estimators=100, 
                                 random_state=0).fit(train_X, train_y)

In [11]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(my_model, random_state= 1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

Weight,Feature
0.1750 ± 0.0848,Goal Scored
0.0500 ± 0.0637,Distance Covered (Kms)
0.0437 ± 0.0637,Yellow Card
0.0187 ± 0.0500,Off-Target
0.0187 ± 0.0637,Free Kicks
0.0187 ± 0.0637,Fouls Committed
0.0125 ± 0.0637,Pass Accuracy %
0.0125 ± 0.0306,Blocked
0.0063 ± 0.0612,Saves
0.0063 ± 0.0250,Ball Possession %


**Interpreting Permutation Importances**

The values towards the top are the most important features, and those towards the bottom matter least.

The first number in each row shows how much model performance decreased with a random shuffling (in this case, using "accuracy" as the performance metric).

Like most things in data science, there is some randomness to the exact performance change from a shuffling a column. We measure the amount of randomness in our permutation importance calculation by repeating the process with multiple shuffles. The number after the ± measures how performance varied from one-reshuffling to the next.

You'll occasionally see negative values for permutation importances. In those cases, the predictions on the shuffled (or noisy) data happened to be more accurate than the real data. This happens when the feature didn't matter (should have had an importance close to 0), but random chance caused the predictions on shuffled data to be more accurate. This is more common with small datasets, like the one in this example, because there is more room for luck/chance.

 - *In our example, the most important feature was **Goals scored**. That seems sensible. Soccer fans may have some intuition about whether the orderings of other variables are surprising or not.*

**[ELI5](https://eli5.readthedocs.io/en/latest/#) is a Python library which allows to visualize and debug various Machine Learning models using unified API. It has built-in support for several ML frameworks and provides a way to explain black-box models.**